In [1]:
import warnings
from glob import glob
from itertools import product
from typing import Callable

import hvplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import seaborn_polars as snl
from catboost import CatBoostClassifier
from dataset import Dataset
from eda import (
    build_barplot_by_column,
    check_distributions_for_similarity,
    draw_overlapping_densities,
)
from lf_processing import handle_dates, scan_file, scan_files
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    StratifiedGroupKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from tqdm.notebook import tqdm

RAND = 0
N_FOLDS = 5
TRAIN_PREFIX = "parquet_files/train/train_"
TEST_PREFIX = "parquet_files/test/test_"
sns.set_theme(style="ticks", font_scale=1.7)
warnings.filterwarnings("ignore")

# Обзор

## Описание задачи

https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/discussion

Задача бинарной классификации для предсказания вероятности дефолта по кредиту (target = 1).

Отсутствие кредитной истории может означать многое, включая молодой возраст или предпочтение наличных денег. Без традиционных данных человеку, у которого практически нет кредитной истории, скорее всего, будет отказано. Поставщики потребительского кредитования должны точно определить, какие клиенты могут погасить кредит, а какие нет, и данные имеют решающее значение. Если бы наука о данных могла помочь лучше прогнозировать возможности погашения долга, кредиты могли бы стать более доступными для тех, кто может получить от них наибольшую выгоду.

В настоящее время поставщики потребительского кредитования используют различные статистические методы и методы машинного обучения для прогнозирования кредитного риска. Эти модели обычно называют картами показателей. В реальном мире поведение клиентов постоянно меняется, поэтому каждую систему показателей необходимо регулярно обновлять, а это требует времени. Стабильность системы показателей в будущем имеет решающее значение, поскольку внезапное падение производительности означает, что кредиты в среднем будут выдаваться худшим клиентам. Суть проблемы заключается в том, что поставщики кредитов не могут обнаружить потенциальные проблемы раньше, чем станут заметны первые сроки погашения этих кредитов. Учитывая время, необходимое для повторной разработки, проверки и внедрения системы показателей, стабильность крайне желательна. Существует компромисс между стабильностью модели и ее производительностью, и перед развертыванием необходимо достичь баланса.

Данная работа по оценке рисков дефолта потенциальных клиентов позволит поставщикам потребительского кредитования принимать больше заявок на кредит. Это может улучшить жизнь людей, которым исторически отказывали в кредите из-за отсутствия кредитной истории.

### Описание таблиц

Этот набор данных содержит большое количество таблиц в результате использования различных источников данных и различных уровней агрегирования данных, используемых при подготовке набора данных. Примечание. Все файлы, перечисленные ниже, существуют в форматах .csv и .parquet.

- Base tables: store the basic information about the observation and case_id. This is a unique identification of every observation and you need to use it to join the other tables to base tables. **train_base.parquet**
- static_0: depth=0, internal data source. **train_static_0_*.parquet**
- static_cb_0: depth=0, external data source. **train_static_cb_0.parquet**
- applprev_1: depth=1, internal data source. **train_applprev_1_*.parquet**
- other_1: depth=1, internal data source. **train_other_1.parquet**
- tax_registry_a_1: depth=1, external data source, Tax registry provider A. **train_tax_registry_a_1.parquet**
- tax_registry_b_1: depth=1, external data source, Tax registry provider B. **train_tax_registry_b_1.parquet**
- tax_registry_c_1: depth=1, external data source, Tax registry provider C. **train_tax_registry_c_1.parquet**
- credit_bureau_a_1: depth=1, external data source, Credit bureau provider A. **train_credit_bureau_a_1_*.parquet**
- credit_bureau_b_1: depth=1, external data source, Credit bureau provider B. **train_credit_bureau_b_1.parquet**
- deposit_1: depth=1, internal data source. **train_deposit_1.parquet**
- person_1: depth=1, internal data source. **train_person_1.parquet**
- debitcard_1: depth=1, internal data source. **train_debitcard_1.parquet**
- applprev_2: depth=2, internal data source. **train_applprev_2.parquet**
- person_2: depth=2, internal data source. **train_person_2.parquet**
- credit_bureau_a_2: depth=2, external data source, Credit bureau provider A. **train_credit_bureau_a_2_*.parquet**
- credit_bureau_b_2: depth=2, external data source, Credit bureau provider B. **train_credit_bureau_b_2_*.parquet**

**Depth values:**
- depth=0 - These are static features directly tied to a specific case_id.
- depth=1 - Each case_id has an associated historical record, indexed by num_group1.
- depth=2 - Each case_id has an associated historical record, indexed by both num_group1 and num_group2.

### Колонки

Special columns:

- case_id - This is the unique identifier for each credit case. You'll need this ID to join relevant tables to the base table.
- date_decision - This refers to the date when a decision was made regarding the approval of the loan.
- WEEK_NUM - This is the week number used for aggregation. In the test sample, WEEK_NUM continues sequentially from the last training value of WEEK_NUM.
- MONTH - This column represents the month and is intended for aggregation purposes.
- target - This is the target value, determined after a certain period based on whether or not the client defaulted on the specific credit case (loan).
- num_group1 - This is an indexing column used for the historical records of case_id in both depth=1 and depth=2 tables.
- num_group2 - This is the second indexing column for depth=2 tables' historical records of case_id. The order of num_group1 and num_group2 is important and will be clarified in feature definitions.

All other raw columns in the tables serve as predictors. Their definitions can be found in the file feature_definitions.csv. For depth=0 tables, predictors can be directly used as features. However, for tables with depth>0, you may need to employ aggregation functions that will condense the historical records associated with each case_id into a single feature. In case num_group1 or num_group2 stands for person index (this is clear with predictor definitions) the zero index has special meaning. When num_groupN=0 it is the applicant (the person who applied for a loan).

Various predictors were transformed, therefore we have the following notation for similar groups of transformations

- **P** - Transform DPD (Days past due)
- **M** - Masking categories
- **A** - Transform amount
- **D** - Transform date
- **T** - Unspecified Transform
- **L** - Unspecified Transform

Полное описание признаков (~500) указано в файле feature_definitions.csv

## Обзор таблиц

Рассмотрим отдельно каждую таблицу.

In [2]:
def show_misses(df: pd.DataFrame) -> None:
    """
    Prints the proportion of gaps for each column in df.

    Parameters
    ----------
    df: pd.DataFrame

    Returns
    -------
    None
    """
    for col in sorted(df.columns):
        n_misses = df[col].isna().sum()
        if n_misses == 0:
            continue
        print("%s = %s %%" % (col, round(n_misses * 100 / df.shape[0], 2)))

In [3]:
def pd_read_files(path: str) -> pd.DataFrame:
    """
    Read into a DataFrame from multiple parquet files.

    Parameters
    ----------
    path: str
        Path to files

    Returns
    -------
    df: pd.DataFrame
    """
    chunks = [pd.read_parquet(x) for x in glob(path)]
    df = pd.concat(chunks)
    df = df.drop_duplicates(subset=["case_id"])
    return df

### base 

In [2]:
df = pd.read_parquet(TRAIN_PREFIX + "base.parquet")
df.describe()

,case_id,MONTH,WEEK_NUM,target
count,1.526659e+06,1.526659e+06,1.526659e+06,1.526659e+06
mean,1.286077e+06,2.019363e+05,4.076904e+01,3.143728e-02
std,7.189466e+05,4.473597e+01,2.379798e+01,1.744964e-01
min,0.000000e+00,2.019010e+05,0.000000e+00,0.000000e+00
25%,7.661975e+05,2.019060e+05,2.300000e+01,0.000000e+00
50%,1.357358e+06,2.019100e+05,4.000000e+01,0.000000e+00
75%,1.739022e+06,2.020010e+05,5.500000e+01,0.000000e+00
max,2.703454e+06,2.020100e+05,9.100000e+01,1.000000e+00


In [3]:
df.describe(include=object)

,date_decision
count,1526659
unique,644
top,2019-11-29
freq,8812


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526659 entries, 0 to 1526658
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   case_id        1526659 non-null  int64 
 1   date_decision  1526659 non-null  object
 2   MONTH          1526659 non-null  int64 
 3   WEEK_NUM       1526659 non-null  int64 
 4   target         1526659 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 58.2+ MB


In [5]:
df

,case_id,date_decision,MONTH,WEEK_NUM,target
0,0,2019-01-03,201901,0,0
1,1,2019-01-03,201901,0,0
2,2,2019-01-04,201901,0,0
3,3,2019-01-03,201901,0,0
4,4,2019-01-04,201901,0,1
...,...,...,...,...,...
1526654,2703450,2020-10-05,202010,91,0
1526655,2703451,2020-10-05,202010,91,0
1526656,2703452,2020-10-05,202010,91,0
1526657,2703453,2020-10-05,202010,91,0


Это базовая таблица к которой другие будут соединяться. Уже можно выдвинуть гипотезу что колонки date_decision, MONTH и WEEK_NUM особой информации не несут, т.к. это относится к дате принятия решения об одобрении кредита. Пропусков нет.
Колонку MONTH уберем, вместо нее из date_decision вычислим месяц, аналогично добавим признак - день недели (weekday_decision).

### depth_0 

#### static_cb_0

In [49]:
df = pd.read_parquet(TRAIN_PREFIX + "static_cb_0.parquet")
df.describe()

,case_id,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,...,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
count,1.500476e+06,1.573290e+05,1.385691e+06,1.385691e+06,1.385691e+06,1.385691e+06,1.385691e+06,1.385691e+06,36514.000000,36514.000000,...,114978.000000,71845.000000,114978.000000,71845.000000,146406.000000,572638.000000,572638.000000,53560.000000,1.385691e+06,1.385691e+06
mean,1.284032e+06,6.416044e+05,1.607715e+00,2.388656e+00,5.177078e-01,4.777066e+00,1.211420e+00,2.860590e+00,0.000082,4.382346,...,10033.556094,17651.732489,6.598027,13.061118,5.714991,5.839291,13199.935970,0.225968,2.688482e+00,2.918342e+00
std,7.160881e+05,9.803273e+05,2.083003e+00,2.891115e+00,8.992377e-01,5.168856e+00,1.655931e+00,3.610966e+00,0.009064,5.815514,...,5455.843604,6871.642301,2.188992,1.855216,1.758117,4.148264,18117.218312,0.976170,3.324546e+00,3.423862e+00
min,3.570000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,...,4.200000,4.400000,1.000000,1.000000,0.000000,0.000000,0.000000,-3.670423,0.000000e+00,0.000000e+00
25%,7.685088e+05,7.853195e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,...,7192.000000,13664.601000,6.000000,12.000000,6.000000,3.000000,3156.400100,-0.227985,0.000000e+00,0.000000e+00
50%,1.361878e+06,3.072824e+05,1.000000e+00,2.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,0.000000,2.000000,...,7553.000000,15765.200000,6.000000,14.000000,6.000000,6.000000,8391.900000,0.371834,2.000000e+00,2.000000e+00
75%,1.737010e+06,8.021141e+05,2.000000e+00,3.000000e+00,1.000000e+00,6.500000e+00,2.000000e+00,4.000000e+00,0.000000,6.000000,...,13464.400000,21840.000000,6.000000,14.000000,6.000000,7.000000,16992.000000,0.971653,4.000000e+00,4.000000e+00
max,2.703454e+06,3.129676e+07,1.090000e+02,1.100000e+02,2.200000e+01,1.150000e+02,4.100000e+01,7.600000e+01,1.000000,57.000000,...,205848.610000,99085.400000,15.000000,16.000000,66.000000,121.000000,476843.400000,2.119132,1.090000e+02,6.200000e+01


In [9]:
df.describe(include=object)

,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,dateofbirth_337D,dateofbirth_342D,description_5085714M,education_1103M,education_88M,maritalst_385M,maritalst_893M,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T
count,136996,114978,71633,607871,1385691,36500,1500476,1500476,1500476,1500476,1500476,673264,720000,660327,224912,53559
unique,8887,372,7565,667,724,664,2,5,5,6,6,3,292,397,207,16
top,2019-09-30,2019-12-13,2005-06-15,1987-05-01,1988-07-01,1986-10-01,a55475b1,a55475b1,a55475b1,a55475b1,a55475b1,DEDUCTION_6,2019-06-28,2019-12-13,2020-09-15,3% - 4%
freq,811,967,3924,1531,3444,120,1316125,859962,1484960,661592,1479303,550387,5252,8735,4266,6445


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500476 entries, 0 to 1500475
Data columns (total 53 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   case_id                  1500476 non-null  int64  
 1   assignmentdate_238D      136996 non-null   object 
 2   assignmentdate_4527235D  114978 non-null   object 
 3   assignmentdate_4955616D  71633 non-null    object 
 4   birthdate_574D           607871 non-null   object 
 5   contractssum_5085716L    157329 non-null   float64
 6   dateofbirth_337D         1385691 non-null  object 
 7   dateofbirth_342D         36500 non-null    object 
 8   days120_123L             1385691 non-null  float64
 9   days180_256L             1385691 non-null  float64
 10  days30_165L              1385691 non-null  float64
 11  days360_512L             1385691 non-null  float64
 12  days90_310L              1385691 non-null  float64
 13  description_5085714M     1500476 non-null 

In [11]:
df

,case_id,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,...,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
0,357,None,None,None,1988-04-01,NaN,None,None,NaN,NaN,...,6.0,6301.4000,None,2019-01-25,None,None,None,NaN,NaN,NaN
1,381,None,None,None,1973-11-01,NaN,None,None,NaN,NaN,...,6.0,4019.6000,None,2019-01-25,None,None,None,NaN,NaN,NaN
2,388,None,None,None,1989-04-01,NaN,1989-04-01,None,6.0,8.0,...,6.0,14548.0000,None,2019-01-28,None,None,None,NaN,3.0,5.0
3,405,None,None,None,1974-03-01,NaN,1974-03-01,None,0.0,0.0,...,6.0,10498.2400,None,2019-01-21,None,None,None,NaN,2.0,0.0
4,409,None,None,None,1993-06-01,NaN,1993-06-01,None,2.0,3.0,...,7.0,6344.8804,None,2019-01-21,None,None,None,NaN,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500471,2703450,None,None,2018-01-11,None,52863.59,1960-01-01,None,0.0,0.0,...,NaN,NaN,None,None,None,2020-10-19,None,NaN,1.0,1.0
1500472,2703451,None,None,2005-06-15,None,324608.52,1950-11-01,None,0.0,0.0,...,NaN,NaN,None,None,None,2020-10-19,None,NaN,1.0,2.0
1500473,2703452,None,None,None,None,102738.76,1977-08-01,None,2.0,2.0,...,NaN,NaN,None,None,None,2020-10-19,None,NaN,0.0,4.0
1500474,2703453,None,None,2008-02-15,None,212683.29,1950-02-01,None,2.0,2.0,...,NaN,NaN,None,None,None,2020-10-17,None,NaN,2.0,1.0


In [50]:
df["description_5085714M"].unique()

array(['a55475b1', '2fc785b2'], dtype=object)

In [14]:
show_misses(df)

assignmentdate_238D = 90.87 %
assignmentdate_4527235D = 92.34 %
assignmentdate_4955616D = 95.23 %
birthdate_574D = 59.49 %
contractssum_5085716L = 89.51 %
dateofbirth_337D = 7.65 %
dateofbirth_342D = 97.57 %
days120_123L = 7.65 %
days180_256L = 7.65 %
days30_165L = 7.65 %
days360_512L = 7.65 %
days90_310L = 7.65 %
firstquarter_103L = 7.65 %
for3years_128L = 97.57 %
for3years_504L = 97.57 %
for3years_584L = 97.57 %
formonth_118L = 97.57 %
formonth_206L = 97.57 %
formonth_535L = 97.57 %
forquarter_1017L = 97.57 %
forquarter_462L = 97.57 %
forquarter_634L = 97.57 %
fortoday_1092L = 97.57 %
forweek_1077L = 97.57 %
forweek_528L = 97.57 %
forweek_601L = 97.57 %
foryear_618L = 97.57 %
foryear_818L = 97.57 %
foryear_850L = 97.57 %
fourthquarter_440L = 7.65 %
numberofqueries_373L = 7.65 %
pmtaverage_3A = 90.43 %
pmtaverage_4527227A = 92.34 %
pmtaverage_4955615A = 95.21 %
pmtcount_4527229L = 92.34 %
pmtcount_4955617L = 95.21 %
pmtcount_693L = 90.24 %
pmtscount_423L = 61.84 %
pmtssum_45A = 61.84 

Для этой таблицы наблюдаем колонки с большим количеством пропусков (более 90 %). Для оптимизации эти колонки можно не учитывать. Такие колонки как 'description_5085714M' где в конце буква 'M' можно закодировать label encoding. Также из описания колонок можно заметить что есть колонки с похожим названием. Например  'description_5085714M'  и description_351M (Categorization of clients by credit bureau) можно агрегировать в одну колонку.

- Колонки assignmentdate, contractssum_5085716L, pmtaverage, pmtcount, riskassesment_302T безнадежны, их удалим. 
- Колонки с датой рождения dateofbirth, birthdate соединим в одну.
- Признаки days*** пока оставляем (см. Гипотезу 1)
- firstquarter_103L, secondquarter_766L, thirdquarter_1082L, fourthquarter_440L - Количество результатов, полученных от кредитного бюро по кварталам. Эти 4 признака объединим в один, получим количество результатов за год (см. Гипотеза 2).
- description_5085714M имеет всего 2 значения  "a55475b1" "2fc785b2". Причем одно из них встречается крайне редко. (см. Гипотезу 3). Этот признак исключаем.
- education_1103M и education_88M закодируем и объединим в одну колонку (см. Гипотезу 4).
- у признаков for3years_128L...foryear_850L наблюдаем 97.57 % пропусков. Эти колонки игнорируем.
- maritalst_385M оставим, закодируем, а maritalst_893M уберем, он неинформативен (см. гипотезу 5).
- requesttype_4525192L, оставим, пропуски заполним 'None' (см. гипотезу 6).

**Сократили количество колонок с 53 до 15 на первом этапе**

#### static_0_*

In [26]:
df = pd_read_files(TRAIN_PREFIX + "static_0_*.parquet")
df.describe()

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
count,1.526659e+06,1.108481e+06,9.655350e+05,1.526659e+06,1.526655e+06,1.526659e+06,1.526659e+06,1.526659e+06,1.526659e+06,1.526659e+06,...,1.394771e+06,1.345537e+06,1.303419e+06,1.526659e+06,1.526659e+06,1.081339e+06,6.860130e+05,1.526656e+06,1.526655e+06,352448.000000
mean,1.286077e+06,7.923365e-02,5.595833e+04,4.039207e+03,1.435775e+03,2.161583e-05,1.381520e-01,4.245100e-01,1.073578e+00,2.687129e-01,...,8.049350e-03,3.052462e-02,3.446497e+04,1.628451e-01,1.615547e+00,2.768127e+04,2.830974e+04,1.968312e+04,9.223817e+04,10411.377565
std,7.189466e+05,1.120652e+01,7.161417e+04,3.006608e+03,2.807021e+03,8.133702e-03,4.886632e-01,3.128111e+00,9.715778e+00,2.464199e+00,...,8.935639e-02,1.720258e-01,3.448316e+04,4.502503e-01,2.061152e+00,5.840378e+04,6.050010e+04,5.083603e+04,1.623358e+05,16222.912082
min,0.000000e+00,0.000000e+00,0.000000e+00,8.080000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.504400e+04,-2.504400e+04,0.000000e+00,0.000000e+00,0.214000
25%,7.661975e+05,0.000000e+00,7.419200e+03,1.967600e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.380000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3309.314000
50%,1.357358e+06,0.000000e+00,2.975840e+04,3.151800e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,2.515600e+04,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.597766e+04,6221.200000
75%,1.739022e+06,0.000000e+00,7.630295e+04,5.231400e+03,2.029400e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,4.532000e+04,0.000000e+00,2.000000e+00,2.799400e+04,2.798400e+04,1.349310e+04,1.188157e+05,11685.000000
max,2.703454e+06,4.206000e+03,1.408010e+06,1.060070e+05,8.750000e+04,5.000000e+00,2.800000e+01,7.280000e+02,2.470000e+02,9.000000e+01,...,1.000000e+00,1.000000e+00,7.618674e+05,8.000000e+00,1.020000e+02,1.210629e+06,1.085048e+06,1.210629e+06,4.803504e+07,794899.200000


In [27]:
df.describe(include=object)

,bankacctype_710L,cardtype_51L,credtype_322L,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,disbursementtype_67L,dtlastpmtallstes_4499206D,firstclxcampaign_1125D,firstdatedue_489D,...,lastrepayingdate_696D,lastst_736L,maxdpdinstldate_3546855D,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,previouscontdistrict_112M,twobodfilling_608L,typesuite_864L,validfrom_1069D
count,417030,191691,1526658,692924,133818,639000,1525793,549540,667445,1044556,...,2377,1221522,700659,1468084,1468084,9329,1526659,1526568,405154,151773
unique,1,2,3,1404,5175,5231,3,4464,1248,5210,...,539,11,5287,1,1,596,223,2,1,618
top,CA,INSTANT,COL,2008-04-03,2020-06-15,2019-09-15,SBA,2019-09-16,2016-01-31,2018-02-15,...,2019-12-15,D,2019-08-11,OTHER,OTHER,2020-06-10,a55475b1,FO,AL,2019-06-18
freq,417030,189370,988376,140227,355,1677,1138811,13406,229032,2459,...,23,386249,1439,1468084,1468084,353,488428,1481471,405154,15175


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1526659 entries, 0 to 522901
Columns: 168 entries, case_id to validfrom_1069D
dtypes: bool(1), float64(133), int64(1), object(33)
memory usage: 1.9+ GB


In [29]:
df

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,0,NaN,NaN,1917.6000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,None,None
1,1,NaN,NaN,3134.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,None,None
2,2,NaN,NaN,4937.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,AL,None
3,3,NaN,NaN,4643.6000,0.0,0.0,1.0,0.0,2.0,0.0,...,1.0,1.0,NaN,NaN,0.0,0.00,NaN,BO,AL,None
4,4,NaN,NaN,3390.2000,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,AL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522897,2703450,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,428159.66,14346.319,FO,None,None
522898,2703451,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,0.0,...,0.0,3.0,68098.4,68098.4,68098.4,701247.30,40499.805,FO,None,None
522899,2703452,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,24002.00,NaN,BO,None,None
522900,2703453,0.0,197371.58,1195.4000,2827.2,0.0,0.0,36.0,0.0,0.0,...,0.0,4.0,46806.6,46806.6,46806.6,440145.30,5654.400,BO,None,None


In [30]:
show_misses(df)

actualdpdtolerance_344P = 27.39 %
amtinstpaidbefduel24m_4187115A = 36.76 %
annuitynextmonth_57A = 0.0 %
avgdbddpdlast24m_3658932P = 40.17 %
avgdbddpdlast3m_4187120P = 62.11 %
avgdbdtollast24m_4525197P = 63.72 %
avgdpdtolclosure24_3658938P = 30.6 %
avginstallast24m_3658937A = 40.93 %
avglnamtstart24m_4525187A = 89.36 %
avgmaxdpdlast9m_3716943P = 49.59 %
avgoutstandbalancel6m_4187114A = 55.1 %
avgpmtlast12m_4525200A = 67.27 %
bankacctype_710L = 72.68 %
cardtype_51L = 87.44 %
clientscnt_136L = 99.97 %
cntincpaycont9m_3716944L = 29.82 %
cntpmts24_3658933L = 30.18 %
commnoinclast6m_3546845L = 22.49 %
credtype_322L = 0.0 %
currdebt_22A = 0.0 %
currdebtcredtyperange_828A = 0.0 %
datefirstoffer_1144D = 54.61 %
datelastinstal40dpd_247D = 91.23 %
datelastunpaid_3546854D = 58.14 %
daysoverduetolerancedd_3976961L = 29.65 %
disbursementtype_67L = 0.06 %
dtlastpmtallstes_4499206D = 64.0 %
eir_270L = 12.5 %
equalitydataagreement_891L = 94.89 %
equalityempfrom_62L = 97.52 %
firstclxcampaign_1125D = 56

Довольно объемная таблица с 168 колонками. Всего несколько из них с пропусками более 90 %.

- amtinstpaidbefduel24m_4187115A (Количество платежей, выплаченных досрочно за последние 24 месяца) заполним пропуски нулями. (см. гипотезу 7)
- **A** (amount) пропуски заполним медианой.
- annuity_780A информативный признак (см. гипотезу 8).
- annuitynextmonth_57A пропуски заполним 0.
- 6 колонок applicationcnt объединим по смыслу. 4 их них - количество запросов от клиента. 2 остальные - количество запросов от клиентов с подобным работодателем.
- колонки avgdbddpdlast24m_3658932P и avgdbddpdlast3m_4187120P объединим
- колонки avgdpdtolclosure24_3658938P и avgmaxdpdlast9m_3716943P объединим
- 13 колонок clientscnt_ объединим, т.к. у них схожий смысл
- 3 колонки clientscnt объединим, т.к. у них схожий смысл
- currdebt_22A и currdebtcredtyperange_828A по смыслу похожи, объединим (Current amount of debt of the applicant.)
- firstclxcampaign_1125D и datefirstoffer_1144D объединим.
- isbidproduct_1095L и isbidproductrequest_292L объединим (Флаг, указывающий, является ли продукт перекрестной продажей)
- mastercontrelectronic_519L и mastercontrexist_109L не содержат положительных значений, только 0 и null. Этот признак неинформативен.
- maxdbddpdlast1m_3658939P, maxdbddpdtollast12m_3658940P, maxdbddpdtollast6m_4187119P объединим
- paytype неинформативны

**Сократили количество колонок с 168 до 88 на первом этапе.**

### depth_1 

#### applprev_1

In [32]:
df = pd_read_files(TRAIN_PREFIX + "applprev_1_*.parquet")
df.describe()

,case_id,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,...,downpmt_134A,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
count,1.221522e+06,1.221505e+06,1.217913e+06,214791.000000,613392.000000,58212.000000,1.219213e+06,5.821200e+04,58212.000000,58212.000000,...,1.219212e+06,1.221522e+06,319779.000000,1.149017e+06,815555.000000,1.221522e+06,915763.000000,987676.000000,2.182620e+05,987676.000000
mean,1.426493e+06,1.825944e-02,1.706922e+03,17968.499237,0.967184,20050.164617,8.427379e+03,-2.900358e+03,-6137.357323,0.550608,...,5.059082e+02,2.400366e-02,0.230293,3.775816e+04,19.353119,2.172528e+00,4258.545625,13.828349,7.644025e+08,13.828349
std,7.274805e+05,7.673556e+00,1.959244e+03,28411.617756,1.262145,25434.890344,2.251972e+04,4.130169e+04,16551.727521,3.319402,...,2.751810e+03,1.530605e-01,0.421021,3.036225e+04,196.247125,3.145552e+00,15353.215365,8.651554,2.016545e+07,8.651554
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,-114086.000000,0.000000e+00,-2.902651e+05,-309628.030000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,3.000000,5.403559e+08,3.000000
25%,9.489082e+05,0.000000e+00,0.000000e+00,1.000000,0.000000,3.480000,0.000000e+00,0.000000e+00,-0.071000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,1.600000e+04,0.000000,0.000000e+00,0.000000,6.000000,7.427542e+08,6.000000
50%,1.509926e+06,0.000000e+00,1.358000e+03,1000.000000,1.000000,12558.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,3.000000e+04,0.000000,1.000000e+00,0.000000,12.000000,7.605669e+08,12.000000
75%,1.815307e+06,0.000000e+00,2.353000e+03,28151.000000,2.000000,30390.662750,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,5.000000e+04,0.000000,3.000000e+00,0.000000,18.000000,7.805058e+08,18.000000
max,2.703454e+06,4.206000e+03,1.030000e+05,200000.000000,20.000000,519966.000000,4.000000e+05,7.988198e+06,101840.000000,147.000000,...,4.204000e+05,1.000000e+00,1.000000,1.996000e+05,4362.000000,1.900000e+01,614749.800000,62.000000,8.006087e+08,62.000000


In [33]:
df.describe(include=object)

,approvaldate_319D,cancelreason_3545846M,creationdate_885D,credacc_status_367L,credtype_587L,dateactivated_425D,district_544M,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,firstnonzeroinstldate_307D,inittransactioncode_279L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,status_219L
count,837653,1221522,1221522,58212,1219212,804815,1221522,352497,590784,1221522,676798,870312,899957,1219212,1221522,1221522,1221522,1221522,1221522
unique,5397,69,5399,6,3,4162,720,2189,2197,6,7610,5,5108,3,9,2937,18,12,11
top,2018-12-07,a55475b1,2018-12-07,AC,COL,2019-01-31,a55475b1,2019-09-16,2019-09-16,P97_36_170,2017-01-15,MARRIED,2019-02-15,POS,a55475b1,a55475b1,a55475b1,a55475b1,K
freq,1445,953086,1922,31924,711976,1832,131933,9057,10598,515030,22040,628048,2098,882051,761412,1210531,983806,1027974,557855


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1221522 entries, 0 to 2638293
Data columns (total 41 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   case_id                      1221522 non-null  int64  
 1   actualdpd_943P               1221505 non-null  float64
 2   annuity_853A                 1217913 non-null  float64
 3   approvaldate_319D            837653 non-null   object 
 4   byoccupationinc_3656910L     214791 non-null   float64
 5   cancelreason_3545846M        1221522 non-null  object 
 6   childnum_21L                 613392 non-null   float64
 7   creationdate_885D            1221522 non-null  object 
 8   credacc_actualbalance_314A   58212 non-null    float64
 9   credacc_credlmt_575A         1219213 non-null  float64
 10  credacc_maxhisbal_375A       58212 non-null    float64
 11  credacc_minhisbal_90A        58212 non-null    float64
 12  credacc_status_367L          58212 non-nul

In [35]:
df

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,2,0.0,640.2,None,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
2,3,0.0,6140.0,None,NaN,P94_109_143,NaN,2019-01-07,NaN,0.0,...,0,NaN,12.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,12.0
3,4,0.0,2556.6,None,NaN,P24_27_36,NaN,2019-01-08,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,24.0
4,5,0.0,NaN,None,NaN,P85_114_140,NaN,2019-01-16,NaN,NaN,...,0,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,NaN
5,6,0.0,1110.4,None,1.0,a55475b1,0.0,2014-11-18,NaN,0.0,...,2,0.0,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638262,2703450,0.0,0.0,2019-01-07,NaN,a55475b1,NaN,2019-01-07,NaN,0.0,...,1,0.0,NaN,P46_145_78,a55475b1,a55475b1,a55475b1,760714940.0,A,NaN
2638275,2703451,0.0,0.0,2019-09-30,NaN,a55475b1,NaN,2019-09-30,NaN,0.0,...,0,0.0,NaN,P46_145_78,a55475b1,a55475b1,a55475b1,780594500.0,A,NaN
2638281,2703452,0.0,1501.0,2019-08-19,NaN,a55475b1,NaN,2019-08-19,NaN,0.0,...,1,0.0,3.0,P177_117_192,a55475b1,a55475b1,a55475b1,NaN,K,3.0
2638284,2703453,0.0,0.0,2019-12-18,NaN,a55475b1,NaN,2019-12-18,NaN,0.0,...,0,0.0,NaN,P46_145_78,a55475b1,a55475b1,a55475b1,780825900.0,A,NaN


In [36]:
show_misses(df)

actualdpd_943P = 0.0 %
annuity_853A = 0.3 %
approvaldate_319D = 31.43 %
byoccupationinc_3656910L = 82.42 %
childnum_21L = 49.78 %
credacc_actualbalance_314A = 95.23 %
credacc_credlmt_575A = 0.19 %
credacc_maxhisbal_375A = 95.23 %
credacc_minhisbal_90A = 95.23 %
credacc_status_367L = 95.23 %
credacc_transactions_402L = 95.23 %
credamount_590A = 0.19 %
credtype_587L = 0.19 %
currdebt_94A = 24.82 %
dateactivated_425D = 34.11 %
downpmt_134A = 0.19 %
dtlastpmt_581D = 71.14 %
dtlastpmtallstes_3545839D = 51.64 %
employedfrom_700D = 44.59 %
familystate_726L = 28.75 %
firstnonzeroinstldate_307D = 26.32 %
inittransactioncode_279L = 0.19 %
isdebitcard_527L = 73.82 %
mainoccupationinc_437A = 5.94 %
maxdpdtolerance_577P = 33.23 %
outstandingdebt_522A = 25.03 %
pmtnum_8L = 19.14 %
revolvingaccount_394A = 82.13 %
tenor_203L = 19.14 %


В этой таблице плохо заполнены 5 колонок:
- credacc_actualbalance_314A: **Actual balance on credit account.**
- credacc_maxhisbal_375A: **Maximal historical balance of previous credit account.**
- credacc_minhisbal_90A: **Minimum historical balance of previous credit accounts.**
- credacc_status_367L: **Account status of previous credit applications.**
- credacc_transactions_402L: **Number of transactions made with the previous credit account of the applicant.**


Для уровней 1 и 2 все строки будем группировать по case_id, дату и числовые признаки брать максимальные, а строковые брать по максимальной дате creationdate_885D, т.к. эта дата полностью заполнена и соответствует историческим данным (по определению из описания к датасету).
Для этой таблицы сократим количество колонок с 41 до 32

#### other_1

In [53]:
df = pd.read_parquet(TRAIN_PREFIX + "other_1.parquet")
df.describe()

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1
count,5.110900e+04,5.110900e+04,5.110900e+04,5.110900e+04,5.110900e+04,5.110900e+04,51109.0
mean,1.419514e+06,7.552902e+03,7.462384e+03,9.967413e+03,2.949396e+03,3.586875e+03,0.0
std,9.245095e+05,3.462571e+04,3.506529e+04,8.939342e+04,4.146773e+04,4.827494e+04,0.0
min,4.380100e+04,0.000000e+00,0.000000e+00,-3.357180e+05,0.000000e+00,0.000000e+00,0.0
25%,2.422410e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,1.811468e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.800000e+00,0.0
75%,1.916206e+06,8.000000e+03,7.740000e+03,2.880000e+02,0.000000e+00,5.400000e+00,0.0
max,2.703453e+06,4.957852e+06,5.168004e+06,4.256314e+06,4.180150e+06,4.622918e+06,0.0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51109 entries, 0 to 51108
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   case_id                      51109 non-null  int64  
 1   amtdebitincoming_4809443A    51109 non-null  float64
 2   amtdebitoutgoing_4809440A    51109 non-null  float64
 3   amtdepositbalance_4809441A   51109 non-null  float64
 4   amtdepositincoming_4809444A  51109 non-null  float64
 5   amtdepositoutgoing_4809442A  51109 non-null  float64
 6   num_group1                   51109 non-null  int64  
dtypes: float64(5), int64(2)
memory usage: 2.7 MB


In [55]:
df

,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A,num_group1
0,43801,12466.6010,12291.2000,914.2000,0.0,304.80002,0
1,43991,3333.4001,3273.4001,0.0000,0.0,0.00000,0
2,44001,10000.0000,10000.0000,0.0000,0.0,0.00000,0
3,44053,0.0000,0.0000,2586.4001,0.0,88.80000,0
4,44130,63.8000,60.8000,0.0000,0.0,0.00000,0
...,...,...,...,...,...,...,...
51104,2703443,344.6000,343.4000,0.0000,0.0,0.00000,0
51105,2703448,0.0000,0.0000,2184.2000,0.0,18.00000,0
51106,2703450,0.0000,0.0000,0.0000,0.0,0.00000,0
51107,2703451,27500.0000,27477.6000,0.0000,0.0,0.00000,0


В этой таблице пропусков не наблюдаем, но если соединить с базовой таблицей, то процент пропусков будет 96.65 %, т.к. тут всего 51109 строк, а в базовой 1,5 млн. Получается что вся эта таблица слишком мала, ее игнорируем.

#### tax_registry_*_1

##### tax_registry_a_1

In [255]:
df = pd_read_files(TRAIN_PREFIX + "tax_registry_a_1.parquet")
df.describe()

,case_id,amount_4527230A,num_group1
count,4.579340e+05,457934.000000,457934.000000
mean,1.324374e+06,1406.018343,2.171088
std,6.484756e+05,2057.801241,2.769530
min,2.863100e+04,0.000000,0.000000
25%,8.702362e+05,356.600000,0.000000
50%,1.559490e+06,850.000000,1.000000
75%,1.722924e+06,1630.000000,4.000000
max,2.702290e+06,42500.000000,69.000000


In [256]:
df.describe(include=object)

,name_4527232M,recorddate_4527225D
count,457934,457934
unique,111803,397
top,5e180ef0,2019-12-14
freq,30860,6726


In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457934 entries, 0 to 3275767
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   case_id              457934 non-null  int64  
 1   amount_4527230A      457934 non-null  float64
 2   name_4527232M        457934 non-null  object 
 3   num_group1           457934 non-null  int64  
 4   recorddate_4527225D  457934 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 21.0+ MB


In [258]:
df

,case_id,amount_4527230A,name_4527232M,num_group1,recorddate_4527225D
0,28631,711.0000,f980a1ea,3,2019-09-13
4,28632,400.0000,5f9b74f5,6,2019-09-13
14,28633,151.6000,5e180ef0,0,2019-09-13
15,28635,5461.8003,52c166dc,1,2019-09-13
17,28636,850.0000,5574ff1b,5,2019-09-13
...,...,...,...,...,...
3275750,2697303,79.6000,5e180ef0,0,2020-09-21
3275751,2700297,159.0000,5e180ef0,0,2020-10-05
3275752,2701074,1211.8000,35d8278b,0,2020-10-07
3275758,2701515,356.0000,5e180ef0,7,2020-10-09


In [259]:
show_misses(df)

##### tax_registry_b_1

In [260]:
df = pd_read_files(TRAIN_PREFIX + "tax_registry_b_1.parquet")
df.describe()

,case_id,amount_4917619A,num_group1
count,1.507320e+05,150732.000000,150732.00000
mean,1.461693e+06,11096.602228,2.60108
std,7.061013e+05,15447.710478,3.15270
min,4.943500e+04,0.000000,0.00000
25%,9.973408e+05,2667.500000,0.00000
50%,1.853474e+06,6885.000000,2.00000
75%,1.908226e+06,13543.200000,4.00000
max,2.703452e+06,344250.000000,69.00000


In [261]:
df.describe(include=object)

,deductiondate_4917603D,name_4917606M
count,150732,150732
unique,259,39977
top,2020-04-03,5e180ef0
freq,2895,18052


In [262]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150732 entries, 0 to 1107927
Data columns (total 5 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   case_id                 150732 non-null  int64  
 1   amount_4917619A         150732 non-null  float64
 2   deductiondate_4917603D  150732 non-null  object 
 3   name_4917606M           150732 non-null  object 
 4   num_group1              150732 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 6.9+ MB


In [263]:
df

,case_id,amount_4917619A,deductiondate_4917603D,name_4917606M,num_group1
0,49435,6885.0,2019-10-16,6b730375,0
11,49490,8646.0,2020-01-02,6a3c1a8a,3
17,49526,29855.0,2020-03-25,926adc86,1
19,49563,780.8,2019-10-21,9288fd65,1
30,49576,4587.0,2019-10-01,f4f3200b,1
...,...,...,...,...,...
1107902,2703443,29241.6,2020-08-05,6d3e6416,4
1107908,2703445,10110.4,2020-05-06,614a1950,0
1107913,2703448,8100.0,2020-04-28,ded265ee,0
1107919,2703449,179.8,2020-05-26,5e180ef0,3


##### tax_registry_c_1

In [264]:
df = pd_read_files(TRAIN_PREFIX + "tax_registry_c_1.parquet")
df.describe()

,case_id,num_group1,pmtamount_36A
count,4.822650e+05,482265.0,482265.000000
mean,1.140433e+06,0.0,2482.728235
std,6.544754e+05,0.0,3388.100784
min,3.570000e+02,0.0,0.000000
25%,6.919160e+05,0.0,850.000000
50%,1.285914e+06,0.0,1500.000000
75%,1.463552e+06,0.0,2895.226000
max,2.629815e+06,0.0,42500.000000


In [265]:
df.describe(include=object)

,employername_160M,processingdate_168D
count,482265,482265
unique,111163,323
top,5e180ef0,2019-04-02
freq,25797,6813


In [266]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482265 entries, 0 to 3343789
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   case_id              482265 non-null  int64  
 1   employername_160M    482265 non-null  object 
 2   num_group1           482265 non-null  int64  
 3   pmtamount_36A        482265 non-null  float64
 4   processingdate_168D  482265 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 22.1+ MB


In [267]:
df

,case_id,employername_160M,num_group1,pmtamount_36A,processingdate_168D
0,357,c91b12ff,0,1200.0000,2019-01-04
6,381,8026f2a0,0,665.8000,2018-12-27
12,388,34cb7f48,0,2200.0000,2019-01-07
18,405,6771878b,0,1649.8000,2018-12-28
24,409,1d197dc0,0,1067.6000,2019-01-02
...,...,...,...,...,...
3343767,2629807,5e180ef0,0,732.4000,2019-10-22
3343775,2629808,52b3209c,0,3409.0000,2019-10-22
3343784,2629809,72f39049,0,977.0000,2019-10-22
3343787,2629812,6069ceca,0,850.0000,2019-10-22


Видим что в таблицах есть колонки с разными именами но смысл у них один. Объединим колонки с датой, amount и employername. Удалим num_group1, т.к. информации он не несет. После соединения с базовой таблицей доля пропусков = 70%. Условно границу взяли как раз 70%. Пока эти признаки оставим.

#### credit_bureau_a_1

In [85]:
df = pd_read_files(TRAIN_PREFIX + "credit_bureau_a_1_*.parquet")
df.describe()

,case_id,annualeffectiverate_199L,annualeffectiverate_63L,contractsum_5085717L,credlmt_230A,credlmt_935A,debtoutstand_525A,debtoverdue_47A,dpdmax_139P,dpdmax_757P,...,prolongationcount_1120L,prolongationcount_599L,residualamount_488A,residualamount_856A,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
count,1.386273e+06,288742.000000,168349.000000,1.101540e+05,2.723750e+05,5.476150e+05,7.706460e+05,7.706460e+05,960625.000000,973583.000000,...,60309.000000,12670.000000,270639.000000,5.447160e+05,7.188870e+05,4.158290e+05,6.922190e+05,651075.000000,6.922190e+05,6.510750e+05
mean,1.319438e+06,209.864407,57.389328,1.871034e+05,2.490995e+04,5.735387e+04,1.207301e+05,1.702647e+03,18.232821,82.415030,...,0.473744,0.648303,1.428054,2.566006e+04,8.882309e+04,2.007868e+05,1.350283e+03,62.664324,1.276799e+05,1.360099e+01
std,7.190419e+05,3289.892562,1360.839693,4.126750e+05,5.718038e+05,1.777288e+06,2.243624e+06,1.358844e+05,163.330735,447.313437,...,1.768691,1.958421,540.422626,1.079276e+05,9.179988e+05,2.325643e+06,7.139037e+04,3777.782152,2.163461e+06,2.921273e+03
min,3.880000e+02,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-8.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.740000e+01,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
25%,7.983070e+05,0.120000,0.400000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,1.286410e+04,2.839560e+04,0.000000e+00,0.000000,7.828395e+03,0.000000e+00
50%,1.412385e+06,21.900000,21.870000,4.574350e+04,4.000000e+03,1.000000e+04,2.724690e+04,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.665605e+03,2.569820e+04,6.115600e+04,0.000000e+00,0.000000,3.666611e+04,0.000000e+00
75%,1.760323e+06,38.687500,38.530000,1.824709e+05,2.740000e+04,4.000000e+04,9.975999e+04,0.000000e+00,0.000000,3.000000,...,0.000000,1.000000,0.000000,2.101984e+04,5.198000e+04,1.411466e+05,0.000000e+00,0.000000,1.118721e+05,0.000000e+00
max,2.703454e+06,91250.000000,91250.000000,7.270890e+06,2.440000e+08,7.000000e+08,1.688618e+09,7.069518e+07,4808.000000,84575.000000,...,106.000000,58.000000,239912.920000,6.600000e+07,6.350000e+08,1.391240e+09,2.668111e+07,433225.000000,1.688618e+09,2.030793e+06


In [86]:
df.describe(include=object)

,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,description_351M,...,lastupdate_388D,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,overdueamountmax2date_1002D,overdueamountmax2date_1142D,purposeofcred_426M,purposeofcred_874M,refreshdate_3813885D,subjectrole_182M,subjectrole_93M
count,1386273,1386273,1386273,1386273,963444,990093,990098,963444,981558,1386273,...,990081,353422,271909,354102,275096,1386273,1386273,926949,1386273,1386273
unique,11,292,43,195,8302,9153,6182,5016,5889,12,...,4895,4660,2394,4516,2711,18,23,635,8,9
top,ea6782cc,ea6782cc,7241344e,7241344e,2021-10-14,2019-09-17,2019-01-02,2019-01-11,2011-08-12,a55475b1,...,2007-09-25,2007-07-31,2019-03-26,2011-10-06,2019-03-26,60c73645,60c73645,2019-01-03,a55475b1,a55475b1
freq,882792,633762,933020,915815,3013,3386,760,1576,13501,1366793,...,15225,14654,2712,8451,2543,735270,402059,38316,694054,735198


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1386273 entries, 0 to 2079313
Data columns (total 79 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   case_id                          1386273 non-null  int64  
 1   annualeffectiverate_199L         288742 non-null   float64
 2   annualeffectiverate_63L          168349 non-null   float64
 3   classificationofcontr_13M        1386273 non-null  object 
 4   classificationofcontr_400M       1386273 non-null  object 
 5   contractst_545M                  1386273 non-null  object 
 6   contractst_964M                  1386273 non-null  object 
 7   contractsum_5085717L             110154 non-null   float64
 8   credlmt_230A                     272375 non-null   float64
 9   credlmt_935A                     547615 non-null   float64
 10  dateofcredend_289D               963444 non-null   object 
 11  dateofcredend_353D               990093 non-null  

In [88]:
df

,case_id,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,credlmt_230A,credlmt_935A,...,residualamount_488A,residualamount_856A,subjectrole_182M,subjectrole_93M,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
0,388,NaN,NaN,ea6782cc,a55475b1,7241344e,a55475b1,NaN,NaN,135806.0,...,NaN,114325.805,a55475b1,a55475b1,NaN,NaN,NaN,NaN,NaN,NaN
11,405,NaN,NaN,ea6782cc,a55475b1,7241344e,a55475b1,NaN,NaN,0.0,...,NaN,0.000,ab3c25cf,a55475b1,NaN,NaN,0.0,NaN,224467.500,NaN
22,409,NaN,48.08,ea6782cc,a55475b1,7241344e,a55475b1,NaN,NaN,NaN,...,NaN,NaN,a55475b1,a55475b1,NaN,4180.0,NaN,NaN,NaN,NaN
33,410,NaN,NaN,ea6782cc,a55475b1,7241344e,a55475b1,NaN,NaN,0.0,...,NaN,0.000,ab3c25cf,ab3c25cf,NaN,NaN,0.0,0.0,0.000,0.0
44,411,NaN,NaN,ea6782cc,a55475b1,7241344e,a55475b1,NaN,NaN,160000.0,...,NaN,108604.690,ab3c25cf,ab3c25cf,NaN,NaN,0.0,0.0,347346.880,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2079269,2703450,NaN,NaN,ea6782cc,9158339f,7241344e,7241344e,52863.59,NaN,NaN,...,NaN,NaN,ab3c25cf,ab3c25cf,7700.0,60602.0,0.0,0.0,10572.718,0.0
2079281,2703451,NaN,NaN,ea6782cc,ea6782cc,7241344e,7241344e,324608.52,NaN,NaN,...,NaN,NaN,ab3c25cf,ab3c25cf,75092.0,113980.0,0.0,0.0,64921.707,0.0
2079290,2703452,NaN,NaN,ea6782cc,ea6782cc,7241344e,7241344e,0.00,4000.0,0.0,...,0.0,0.000,ab3c25cf,ab3c25cf,NaN,NaN,0.0,0.0,20547.752,0.0
2079299,2703453,NaN,NaN,ea6782cc,62332675,7241344e,7241344e,212683.29,0.0,NaN,...,0.0,NaN,ab3c25cf,ab3c25cf,NaN,51996.0,0.0,0.0,42536.660,0.0


In [89]:
show_misses(df)

annualeffectiverate_199L = 79.17 %
annualeffectiverate_63L = 87.86 %
contractsum_5085717L = 92.05 %
credlmt_230A = 80.35 %
credlmt_935A = 60.5 %
dateofcredend_289D = 30.5 %
dateofcredend_353D = 28.58 %
dateofcredstart_181D = 28.58 %
dateofcredstart_739D = 30.5 %
dateofrealrepmt_138D = 29.19 %
debtoutstand_525A = 44.41 %
debtoverdue_47A = 44.41 %
dpdmax_139P = 30.7 %
dpdmax_757P = 29.77 %
dpdmaxdatemonth_442T = 29.77 %
dpdmaxdatemonth_89T = 30.7 %
dpdmaxdateyear_596T = 30.7 %
dpdmaxdateyear_896T = 29.77 %
instlamount_768A = 60.71 %
instlamount_852A = 86.61 %
interestrate_508L = 99.33 %
lastupdate_1112D = 30.5 %
lastupdate_388D = 28.58 %
monthlyinstlamount_332A = 30.72 %
monthlyinstlamount_674A = 36.13 %
nominalrate_281L = 75.81 %
nominalrate_498L = 69.51 %
numberofcontrsvalue_258L = 50.07 %
numberofcontrsvalue_358L = 53.03 %
numberofinstls_229L = 48.23 %
numberofinstls_320L = 70.01 %
numberofoutstandinstls_520L = 48.17 %
numberofoutstandinstls_59L = 70.01 %
numberofoverdueinstlmax_1039L

- Колонки 'contractsum', 'description', 'dpdmaxdatemonth','dpdmaxdateyear', 'overdueamountmaxdatemonth', 'overdueamountmaxdateyear', 'prolongationcount', 'refreshdate' при первом просмотре оказались неинформативны, их уберем.
На первом этапе сократили количество колонок 79 до 61. Большинство из оставшихся далее будут объединяться с колонками из других таблиц.

#### credit_bureau_b_1

In [90]:
df = pd.read_parquet(TRAIN_PREFIX + "credit_bureau_b_1.parquet")
df.describe()

,case_id,amount_1115A,credlmt_1052A,credlmt_228A,credlmt_3940954A,credquantity_1099L,credquantity_984L,debtpastduevalue_732A,debtvalue_227A,dpd_550P,...,overdueamountmax_950A,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,pmtdaysoverdue_1135P,pmtnumpending_403L,residualamount_1093A,residualamount_127A,residualamount_3940956A,totalamount_503A,totalamount_881A
count,8.579100e+04,4.368100e+04,2.758100e+04,1.613000e+04,3.821800e+04,53018.00000,46228.000000,8.121700e+04,4.368100e+04,5.301800e+04,...,81224.000000,81224.000000,81224.000000,81217.000000,43680.000000,16125.000000,2.758100e+04,3.751900e+04,5.301800e+04,4.622800e+04
mean,1.218998e+06,2.141105e+05,1.789356e+05,5.231752e+04,1.303603e+05,1.54474,4.565220,3.791878e+03,1.651183e+05,2.569675e+04,...,18.251589,6.630676,2018.290825,71.478619,20.264675,0.019969,5.794230e+04,4.301107e+04,2.570089e+05,2.937632e+05
std,6.863320e+05,6.910196e+05,5.274022e+06,1.280825e+05,2.570305e+06,0.86419,5.170388,1.582381e+05,5.504971e+05,9.686434e+05,...,1036.772496,3.494757,1.741405,4664.511050,29.017417,2.535748,1.106215e+05,9.314620e+04,3.876513e+06,1.202831e+06
min,4.670000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.00000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,1.000000,1900.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.272010e+05,2.599800e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.00000,1.000000,0.000000e+00,1.294600e+04,0.000000e+00,...,0.000000,4.000000,2018.000000,0.000000,5.000000,0.000000,0.000000e+00,0.000000e+00,2.000000e+04,2.572170e+04
50%,1.413976e+06,6.000000e+04,3.618400e+04,2.260000e+04,2.000000e+04,1.00000,3.000000,0.000000e+00,3.589341e+04,0.000000e+00,...,0.000000,7.000000,2019.000000,0.000000,11.000000,0.000000,1.446133e+04,7.483786e+03,6.837960e+04,8.444100e+04
75%,1.778253e+06,1.600000e+05,1.217780e+05,6.000000e+04,7.800000e+04,2.00000,6.000000,0.000000e+00,1.108016e+05,0.000000e+00,...,0.400000,10.000000,2019.000000,0.000000,24.000000,0.000000,7.092540e+04,4.132062e+04,2.000000e+05,2.761022e+05
max,2.703436e+06,5.483333e+07,7.968000e+08,4.420000e+06,3.000000e+08,16.00000,146.000000,4.113871e+07,4.161905e+07,2.078238e+08,...,147470.610000,12.000000,2020.000000,663618.000000,300.000000,322.000000,2.187568e+06,2.022909e+06,7.968000e+08,1.390800e+08


In [91]:
df.describe(include=object)

,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,contracttype_653M,credor_3940957M,lastupdate_260D,periodicityofpmts_997L,periodicityofpmts_997M,pmtmethod_731M,purposeofcred_722M,subjectrole_326M,subjectrole_43M
count,85791,81899,81712,85791,85791,85791,81899,2027,82519,85791,85791,85791,85791
unique,10,4075,4537,15,25,151,611,5,10,10,16,6,6
top,ea6782cc,2019-06-28,2021-10-14,7241344e,4257cbed,b619fa46,2019-11-23,Ежемесячные платежи - 30 дней,a55475b1,a55475b1,60c73645,ab3c25cf,ab3c25cf
freq,67530,148,283,74448,41626,31183,1585,2006,41057,38483,60144,50456,44337


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85791 entries, 0 to 85790
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   case_id                         85791 non-null  int64  
 1   amount_1115A                    43681 non-null  float64
 2   classificationofcontr_1114M     85791 non-null  object 
 3   contractdate_551D               81899 non-null  object 
 4   contractmaturitydate_151D       81712 non-null  object 
 5   contractst_516M                 85791 non-null  object 
 6   contracttype_653M               85791 non-null  object 
 7   credlmt_1052A                   27581 non-null  float64
 8   credlmt_228A                    16130 non-null  float64
 9   credlmt_3940954A                38218 non-null  float64
 10  credor_3940957M                 85791 non-null  object 
 11  credquantity_1099L              53018 non-null  float64
 12  credquantity_984L               

In [93]:
df

,case_id,amount_1115A,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,contracttype_653M,credlmt_1052A,credlmt_228A,credlmt_3940954A,...,pmtmethod_731M,pmtnumpending_403L,purposeofcred_722M,residualamount_1093A,residualamount_127A,residualamount_3940956A,subjectrole_326M,subjectrole_43M,totalamount_503A,totalamount_881A
0,467,78000.00,ea6782cc,2016-10-25,2019-10-25,7241344e,4257cbed,NaN,NaN,NaN,...,e914c86c,10.0,96a8fdfe,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN
1,467,NaN,ea6782cc,2011-06-15,2031-06-13,7241344e,724be82a,3000000.0,10000.0,3000000.0,...,a55475b1,NaN,96a8fdfe,0.0,0.000,NaN,fa4f56f1,ab3c25cf,3000000.00,10000.00
2,467,NaN,ea6782cc,2019-01-04,2021-08-04,7241344e,724be82a,NaN,NaN,130365.0,...,a55475b1,NaN,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,78000.00,960000.00
3,1445,12000.00,ea6782cc,2018-12-31,2019-01-29,7241344e,4257cbed,NaN,NaN,NaN,...,dbcbe8f8,1.0,96a8fdfe,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN
4,1445,31400.00,01f63ac8,2018-07-25,2019-12-25,7241344e,4257cbed,NaN,NaN,NaN,...,dbcbe8f8,12.0,60c73645,NaN,NaN,NaN,a55475b1,a55475b1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85786,2703357,731886.06,07b11743,2018-04-17,2023-04-17,7241344e,4257cbed,NaN,NaN,NaN,...,e914c86c,31.0,96a8fdfe,NaN,NaN,NaN,ab3c25cf,ab3c25cf,731886.06,490729.22
85787,2703357,NaN,ea6782cc,2018-11-17,2020-11-17,7241344e,1c9c5356,72000.0,68996.0,72000.0,...,a55475b1,NaN,60c73645,0.0,72000.000,72000.000,ab3c25cf,ab3c25cf,72000.00,68996.00
85788,2703377,NaN,ea6782cc,2014-01-04,2022-01-04,7241344e,1c9c5356,0.0,NaN,0.0,...,a55475b1,NaN,60c73645,NaN,0.000,0.000,ab3c25cf,ab3c25cf,0.00,205540.00
85789,2703436,800000.00,ea6782cc,2017-10-17,2027-10-17,7241344e,4257cbed,NaN,NaN,NaN,...,e914c86c,85.0,60c73645,NaN,NaN,NaN,ab3c25cf,ab3c25cf,800000.00,131932.00


In [94]:
show_misses(df)

amount_1115A = 49.08 %
contractdate_551D = 4.54 %
contractmaturitydate_151D = 4.75 %
credlmt_1052A = 67.85 %
credlmt_228A = 81.2 %
credlmt_3940954A = 55.45 %
credquantity_1099L = 38.2 %
credquantity_984L = 46.12 %
debtpastduevalue_732A = 5.33 %
debtvalue_227A = 49.08 %
dpd_550P = 38.2 %
dpd_733P = 46.12 %
dpdmax_851P = 5.32 %
dpdmaxdatemonth_804T = 5.32 %
dpdmaxdateyear_742T = 5.32 %
installmentamount_644A = 46.12 %
installmentamount_833A = 38.2 %
instlamount_892A = 49.3 %
interesteffectiverate_369L = 88.92 %
interestrateyearly_538L = 66.4 %
lastupdate_260D = 4.54 %
maxdebtpduevalodued_3940955A = 5.32 %
numberofinstls_810L = 49.3 %
overdueamountmax_950A = 5.32 %
overdueamountmaxdatemonth_494T = 5.32 %
overdueamountmaxdateyear_432T = 5.32 %
periodicityofpmts_997L = 97.64 %
periodicityofpmts_997M = 3.81 %
pmtdaysoverdue_1135P = 5.33 %
pmtnumpending_403L = 49.09 %
residualamount_1093A = 81.2 %
residualamount_127A = 67.85 %
residualamount_3940956A = 56.27 %
totalamount_503A = 38.2 %
totala

- Тут пока объединим 'credlmt_1052A', и 'credlmt_3940954A'.
- 'dpd_550P', 'dpd_733P' удалим, т.к. содержат по большей части нули и аномальные значения.
- 'dpdmaxdatemonth', 'dpdmaxdateyear_742T', lastupdate, maxdebtpduevalodued, 'overdueamountmaxdatemonth', 'overdueamountmaxdateyear' неинформативны
- interesteffectiverate_369L и interestrateyearly_538L объединим в одну
- 'residualamount_3940956A' и 'residualamount_3940956A'  объединим в одну колонку.
Так в первом приближении сократим количество признаков с 45 до 35. Оставшиеся колонки далее будем объединять с остальными данными.

#### person_1

In [95]:
df = pd.read_parquet(TRAIN_PREFIX + "person_1.parquet")
df.describe()

,case_id,childnum_185L,contaddr_matchlist_1032L,contaddr_smempladdr_334L,isreference_387L,mainoccupationinc_384A,num_group1,personindex_1023L,persontype_1072L,persontype_792L,remitter_829L,safeguarantyflag_411L
count,2.973991e+06,9907.000000,1526218.0,1.526218e+06,24916.00000,1.526659e+06,2.973991e+06,2.331708e+06,2.967874e+06,2.331708e+06,805049.0,1.526657e+06
mean,1.055196e+06,0.616029,0.0,4.541946e-03,0.50000,5.770748e+04,7.965317e-01,4.383568e-01,2.034862e+00,2.315691e+00,0.0,9.465551e-01
std,7.245714e+05,0.966080,0.0,6.724076e-02,0.50001,3.334830e+04,9.777888e-01,6.596618e-01,1.707171e+00,1.826378e+00,0.0,2.249190e-01
min,0.000000e+00,0.000000,0.0,0.000000e+00,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.0,0.000000e+00
25%,6.373535e+05,0.000000,0.0,0.000000e+00,0.00000,3.600000e+04,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00
50%,8.908170e+05,0.000000,0.0,0.000000e+00,0.50000,5.000000e+04,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00
75%,1.568334e+06,1.000000,0.0,0.000000e+00,1.00000,7.000000e+04,1.000000e+00,1.000000e+00,4.000000e+00,5.000000e+00,0.0,1.000000e+00
max,2.703454e+06,11.000000,0.0,1.000000e+00,1.00000,2.000000e+05,9.000000e+00,6.000000e+00,5.000000e+00,5.000000e+00,0.0,1.000000e+00


In [96]:
df.describe(include=object)

,birth_259D,birthdate_87D,contaddr_district_15M,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,...,language1_981M,maritalst_703L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,role_1084L,role_993L,sex_738L,type_25L
count,1526659,24916,2973991,2973991,2973991,566701,528315,522236,2973991,2973991,...,2973991,11345,2973991,2973991,805049,805942,2967874,24916,1526659,2967874
unique,680,659,975,3530,6,8075,3,24,223,3339,...,3,5,991,3531,10,10,3,1,2,8
top,1988-07-01,1982-04-01,a55475b1,a55475b1,a55475b1,2018-01-15,MORE_FIVE,OTHER,a55475b1,a55475b1,...,a55475b1,MARRIED,a55475b1,a55475b1,SPOUSE,SPOUSE,CL,FULL,F,PRIMARY_MOBILE
freq,3713,197,1449674,1474844,2234573,48503,371321,386837,2448480,2448349,...,1505452,5970,1447423,1515676,152389,152428,1625689,24916,952776,1770812


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973991 entries, 0 to 2973990
Data columns (total 37 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   case_id                    int64  
 1   birth_259D                 object 
 2   birthdate_87D              object 
 3   childnum_185L              float64
 4   contaddr_district_15M      object 
 5   contaddr_matchlist_1032L   float64
 6   contaddr_smempladdr_334L   float64
 7   contaddr_zipcode_807M      object 
 8   education_927M             object 
 9   empl_employedfrom_271D     object 
 10  empl_employedtotal_800L    object 
 11  empl_industry_691L         object 
 12  empladdr_district_926M     object 
 13  empladdr_zipcode_114M      object 
 14  familystate_447L           object 
 15  gender_992L                object 
 16  housetype_905L             object 
 17  housingtype_772L           object 
 18  incometype_1044T           object 
 19  isreference_387L           float64
 20  la

In [98]:
df

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,None,NaN,P88_18_84,0.0,0.0,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,None,None,NaN,CL,None,1.0,F,PRIMARY_MOBILE
1,0,None,None,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,None,...,a55475b1,a55475b1,SPOUSE,None,0.0,EM,None,NaN,None,PHONE
2,0,None,None,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,None,...,a55475b1,a55475b1,COLLEAGUE,SPOUSE,0.0,PE,None,NaN,None,PHONE
3,0,None,None,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,COLLEAGUE,NaN,PE,None,NaN,None,PHONE
4,1,1957-08-01,None,NaN,P103_93_94,0.0,0.0,P176_37_166,P97_36_170,2008-10-29,...,P103_93_94,P176_37_166,None,None,NaN,CL,None,1.0,M,PRIMARY_MOBILE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973986,2703451,None,None,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,None,NaN,CL,None,NaN,None,HOME_PHONE
2973987,2703452,1977-08-01,None,NaN,P133_44_167,0.0,0.0,P59_150_74,a55475b1,None,...,P19_11_176,P11_15_81,None,None,NaN,CL,None,0.0,M,PRIMARY_MOBILE
2973988,2703453,1950-02-01,None,NaN,P123_6_84,0.0,0.0,P46_103_143,a55475b1,None,...,P123_6_84,P46_103_143,None,None,NaN,CL,None,0.0,F,PRIMARY_MOBILE
2973989,2703453,None,None,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,None,NaN,CL,None,NaN,None,HOME_PHONE


In [99]:
show_misses(df)

birth_259D = 48.67 %
birthdate_87D = 99.16 %
childnum_185L = 99.67 %
contaddr_matchlist_1032L = 48.68 %
contaddr_smempladdr_334L = 48.68 %
empl_employedfrom_271D = 80.94 %
empl_employedtotal_800L = 82.24 %
empl_industry_691L = 82.44 %
familystate_447L = 75.5 %
gender_992L = 99.16 %
housetype_905L = 96.61 %
housingtype_772L = 99.67 %
incometype_1044T = 48.67 %
isreference_387L = 99.16 %
mainoccupationinc_384A = 48.67 %
maritalst_703L = 99.62 %
personindex_1023L = 21.6 %
persontype_1072L = 0.21 %
persontype_792L = 21.6 %
relationshiptoclient_415T = 72.93 %
relationshiptoclient_642T = 72.9 %
remitter_829L = 72.93 %
role_1084L = 0.21 %
role_993L = 99.16 %
safeguarantyflag_411L = 48.67 %
sex_738L = 48.67 %
type_25L = 0.21 %


- 'birth_259D',' birthdate_87D' объединим
- empladdr_district_926M и empladdr_zipcode_114M тоже объединим
- 'contaddr_district_15M' и 'contaddr_zipcode_807M', registaddr_district_1083M registaddr_zipcode_184M объединим в одну
- 'contaddr_matchlist_1032L' 'contaddr_smempladdr_334L' объединим
- Плохо заполнены role_993L, isreference_387L, housetype_905L и housingtype_772L
- persontype_1072L и persontype_792L объединим
- Так сократим количество признаков с 37 до 23 в первом приближении.

#### deposit_1

In [100]:
df = pd.read_parquet(TRAIN_PREFIX + "deposit_1.parquet")
df.describe()

,case_id,amount_416A,num_group1
count,1.450860e+05,1.450860e+05,145086.000000
mean,1.466214e+06,8.422304e+03,0.522531
std,8.865290e+05,8.623212e+04,1.620954
min,2.250000e+02,-4.000000e+04,0.000000
25%,6.600410e+05,0.000000e+00,0.000000
50%,1.556939e+06,2.236580e+02,0.000000
75%,2.530539e+06,4.783400e+02,1.000000
max,2.703453e+06,1.221329e+07,64.000000


In [101]:
df.describe(include=object)

,contractenddate_991D,openingdate_313D
count,65404,145086
unique,1524,1579
top,2017-08-03,2014-07-11
freq,295,368


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145086 entries, 0 to 145085
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   case_id               145086 non-null  int64  
 1   amount_416A           145086 non-null  float64
 2   contractenddate_991D  65404 non-null   object 
 3   num_group1            145086 non-null  int64  
 4   openingdate_313D      145086 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 5.5+ MB


In [103]:
df

,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
0,225,0.00000,None,0,2016-08-16
1,331,260.37400,2018-03-18,0,2015-03-19
2,358,0.00000,None,0,2014-09-02
3,390,203.60200,2017-09-30,1,2015-10-01
4,390,223.68001,None,2,2016-06-08
...,...,...,...,...,...
145081,2703430,0.00000,None,7,2016-02-25
145082,2703430,22918.00600,None,8,2016-11-26
145083,2703439,219.98401,None,0,2016-11-25
145084,2703453,0.00000,None,0,2014-08-18


In [104]:
show_misses(df)

contractenddate_991D = 54.92 %


Аналогично как и таблица other_1 эта таблица после соединения с базовой будет имень более 90 % пропусков. Эти данные игнорируем.

#### debitcard_1

In [105]:
df = pd.read_parquet(TRAIN_PREFIX + "debitcard_1.parquet")
df.describe()

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1
count,1.573020e+05,12216.000000,1.108100e+04,11081.000000,157302.000000
mean,1.468784e+06,109.635884,3.849451e+04,4955.383495,0.549306
std,8.883316e+05,949.997458,4.140059e+04,19217.736947,1.639082
min,2.250000e+02,-308.794130,-1.877800e+05,-477.506000,0.000000
25%,6.491730e+05,0.000000,7.878000e+03,0.000000,0.000000
50%,1.560121e+06,0.000000,3.000000e+04,0.000000,0.000000
75%,2.531590e+06,1.053898,6.000000e+04,0.000000,1.000000
max,2.703453e+06,67777.770000,1.161820e+06,390000.000000,65.000000


In [106]:
df.describe(include=object)

,openingdate_857D
count,144591
unique,1578
top,2014-07-11
freq,368


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157302 entries, 0 to 157301
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   case_id                        157302 non-null  int64  
 1   last180dayaveragebalance_704A  12216 non-null   float64
 2   last180dayturnover_1134A       11081 non-null   float64
 3   last30dayturnover_651A         11081 non-null   float64
 4   num_group1                     157302 non-null  int64  
 5   openingdate_857D               144591 non-null  object 
dtypes: float64(3), int64(2), object(1)
memory usage: 7.2+ MB


In [108]:
df

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
0,225,NaN,NaN,NaN,0,2016-08-16
1,331,NaN,NaN,NaN,0,2015-03-19
2,358,NaN,NaN,NaN,0,2014-09-02
3,390,NaN,NaN,NaN,0,2014-07-23
4,390,NaN,NaN,NaN,1,2015-10-01
...,...,...,...,...,...,...
157297,2703430,NaN,NaN,NaN,7,2016-02-25
157298,2703430,NaN,NaN,NaN,8,2016-11-26
157299,2703439,NaN,NaN,NaN,0,2016-11-25
157300,2703453,NaN,NaN,NaN,0,2014-08-18


In [109]:
show_misses(df)

last180dayaveragebalance_704A = 92.23 %
last180dayturnover_1134A = 92.96 %
last30dayturnover_651A = 92.96 %
openingdate_857D = 8.08 %


Аналогично как и таблица other_1 эта таблица после соединения с базовой будет имень более 95 % пропусков. Эти данные игнорируем.

### depth_2 

#### applprev_2

In [111]:
df = pd.read_parquet(TRAIN_PREFIX + "applprev_2.parquet")
df.describe()

,case_id,num_group1,num_group2
count,1.407549e+07,1.407549e+07,1.407549e+07
mean,1.454198e+06,4.551473e+00,7.403148e-01
std,7.875084e+05,4.390422e+00,8.025843e-01
min,2.000000e+00,0.000000e+00,0.000000e+00
25%,1.237440e+06,1.000000e+00,0.000000e+00
50%,1.575626e+06,3.000000e+00,1.000000e+00
75%,1.861301e+06,7.000000e+00,1.000000e+00
max,2.703454e+06,1.900000e+01,1.100000e+01


In [112]:
df.describe(include=object)

,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L
count,13966238,11681431,342083
unique,9,9,6
top,a55475b1,PRIMARY_MOBILE,CANCELLED
freq,13958443,6294386,167031


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14075487 entries, 0 to 14075486
Data columns (total 6 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   case_id                   int64 
 1   cacccardblochreas_147M    object
 2   conts_type_509L           object
 3   credacc_cards_status_52L  object
 4   num_group1                int64 
 5   num_group2                int64 
dtypes: int64(3), object(3)
memory usage: 644.3+ MB


In [114]:
df

,case_id,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group1,num_group2
0,2,None,PRIMARY_MOBILE,None,0,0
1,2,None,EMPLOYMENT_PHONE,None,0,1
2,2,None,PRIMARY_MOBILE,None,1,0
3,2,None,EMPLOYMENT_PHONE,None,1,1
4,3,None,PHONE,None,0,0
...,...,...,...,...,...,...
14075482,2703454,a55475b1,None,None,0,1
14075483,2703454,a55475b1,PRIMARY_MOBILE,None,1,0
14075484,2703454,a55475b1,HOME_PHONE,None,1,1
14075485,2703454,a55475b1,None,None,1,2


In [115]:
show_misses(df)

cacccardblochreas_147M = 0.78 %
conts_type_509L = 17.01 %
credacc_cards_status_52L = 97.57 %


- conts_type_509L учитывать не будем, точно такая же информация есть в признаке type_25L,Contact type of a person и он лучше заполнен.

#### person_2

In [116]:
df = pd.read_parquet(TRAIN_PREFIX + "person_2.parquet")
df.describe()

,case_id,num_group1,num_group2
count,1.643410e+06,1.643410e+06,1.643410e+06
mean,1.264005e+06,1.115425e-01,1.237013e-01
std,6.995455e+05,3.224079e-01,7.612454e-01
min,5.000000e+00,0.000000e+00,0.000000e+00
25%,7.619582e+05,0.000000e+00,0.000000e+00
50%,1.323516e+06,0.000000e+00,0.000000e+00
75%,1.695937e+06,0.000000e+00,0.000000e+00
max,2.703454e+06,4.000000e+00,3.100000e+01


In [117]:
df.describe(include=object)

,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,relatedpersons_role_762T
count,1643410,67674,1643410,1643410,1643410,5757,1643410,28726
unique,508,8,2027,11,10,801,7153,10
top,a55475b1,PERMANENT,a55475b1,a55475b1,a55475b1,2017-01-15,a55475b1,OTHER_RELATIVE
freq,1582872,37338,1576370,1587829,1618686,228,1636009,6211


In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1643410 entries, 0 to 1643409
Data columns (total 11 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   case_id                   1643410 non-null  int64 
 1   addres_district_368M      1643410 non-null  object
 2   addres_role_871L          67674 non-null    object
 3   addres_zip_823M           1643410 non-null  object
 4   conts_role_79M            1643410 non-null  object
 5   empls_economicalst_849M   1643410 non-null  object
 6   empls_employedfrom_796D   5757 non-null     object
 7   empls_employer_name_740M  1643410 non-null  object
 8   num_group1                1643410 non-null  int64 
 9   num_group2                1643410 non-null  int64 
 10  relatedpersons_role_762T  28726 non-null    object
dtypes: int64(3), object(8)
memory usage: 137.9+ MB


In [119]:
df

,case_id,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,num_group1,num_group2,relatedpersons_role_762T
0,5,a55475b1,None,a55475b1,a55475b1,a55475b1,None,a55475b1,0,0,None
1,6,P55_110_32,CONTACT,P10_68_40,P38_92_157,P164_110_33,None,a55475b1,0,0,None
2,6,P55_110_32,PERMANENT,P10_68_40,a55475b1,a55475b1,None,a55475b1,0,1,None
3,6,P204_92_178,CONTACT,P65_136_169,P38_92_157,P164_110_33,None,a55475b1,1,0,OTHER_RELATIVE
4,6,P191_109_75,CONTACT,P10_68_40,P7_147_157,a55475b1,None,a55475b1,1,1,OTHER_RELATIVE
...,...,...,...,...,...,...,...,...,...,...,...
1643405,2703450,a55475b1,None,a55475b1,a55475b1,a55475b1,None,a55475b1,0,0,None
1643406,2703451,a55475b1,None,a55475b1,a55475b1,a55475b1,None,a55475b1,0,0,None
1643407,2703452,a55475b1,None,a55475b1,a55475b1,a55475b1,None,a55475b1,0,0,None
1643408,2703453,a55475b1,None,a55475b1,a55475b1,a55475b1,None,a55475b1,0,0,None


In [120]:
show_misses(df)

addres_role_871L = 95.88 %
empls_employedfrom_796D = 99.65 %
relatedpersons_role_762T = 98.25 %


- Колонки с addres неинформативны. addres_role_871L имеет много пропусков, а в addres_zip_823M и addres_district_368M более 90% встречается только одно значение.
- conts_role_79M, empls_economicalst тоже неинформативны, более 90% приходится на одно значение.
- relatedpersons_role_762T плохо заполнен.

#### credit_bureau_a_2_*

In [122]:
df = pd_read_files(TRAIN_PREFIX + "credit_bureau_a_2_*.parquet")
df.describe()

,case_id,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,num_group1,num_group2,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T
count,1.385288e+06,1.264000e+06,1.005365e+06,1.385288e+06,1.385288e+06,86871.000000,43998.000000,1.264006e+06,1.011554e+06,8.708200e+04,4.414400e+04,1.264006e+06,1.011554e+06
mean,1.319860e+06,8.900942e+04,4.324896e+05,1.356361e-01,6.083212e-03,11.023253,56.025274,2.006024e+00,2.005442e+00,1.379000e+03,4.283630e+03,2.017913e+03,2.011686e+03
std,7.189648e+05,1.583970e+06,2.287902e+07,4.692805e-01,9.021195e-02,121.841748,253.450128,8.687661e-02,8.612403e-02,6.383681e+04,3.567975e+04,8.114835e-01,4.356577e+00
min,3.880000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-8.000000,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.015000e+03,2.000000e+03
25%,7.986868e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.017000e+03,2.007000e+03
50%,1.412778e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.018000e+03,2.012000e+03
75%,1.760539e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.018000e+03,2.016000e+03
max,2.703454e+06,7.714286e+08,3.250000e+09,7.000000e+01,1.400000e+01,3685.000000,4387.000000,1.200000e+01,1.200000e+01,1.736279e+07,2.598532e+06,2.020000e+03,2.020000e+03


In [123]:
df.describe(include=object)

,collater_typofvalofguarant_298M,collater_typofvalofguarant_407M,collaterals_typeofguarante_359M,collaterals_typeofguarante_669M,subjectroles_name_541M,subjectroles_name_838M
count,1385288,1385288,1385288,1385288,1385288,1385288
unique,5,7,15,15,9,9
top,9a0c095e,9a0c095e,c7a5ad39,c7a5ad39,ab3c25cf,ab3c25cf
freq,992875,676994,761746,1228477,996544,1249459


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1385288 entries, 0 to 18722963
Data columns (total 19 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   case_id                          1385288 non-null  int64  
 1   collater_typofvalofguarant_298M  1385288 non-null  object 
 2   collater_typofvalofguarant_407M  1385288 non-null  object 
 3   collater_valueofguarantee_1124L  1264000 non-null  float64
 4   collater_valueofguarantee_876L   1005365 non-null  float64
 5   collaterals_typeofguarante_359M  1385288 non-null  object 
 6   collaterals_typeofguarante_669M  1385288 non-null  object 
 7   num_group1                       1385288 non-null  int64  
 8   num_group2                       1385288 non-null  int64  
 9   pmts_dpd_1073P                   86871 non-null    float64
 10  pmts_dpd_303P                    43998 non-null    float64
 11  pmts_month_158T                  1264006 non-null

In [125]:
df

,case_id,collater_typofvalofguarant_298M,collater_typofvalofguarant_407M,collater_valueofguarantee_1124L,collater_valueofguarantee_876L,collaterals_typeofguarante_359M,collaterals_typeofguarante_669M,num_group1,num_group2,pmts_dpd_1073P,pmts_dpd_303P,pmts_month_158T,pmts_month_706T,pmts_overdue_1140A,pmts_overdue_1152A,pmts_year_1139T,pmts_year_507T,subjectroles_name_541M,subjectroles_name_838M
0,388,8fd95e4b,a55475b1,0.0,NaN,a55475b1,c7a5ad39,0,0,NaN,NaN,2.0,NaN,NaN,NaN,2018.0,NaN,a55475b1,ab3c25cf
36,405,9a0c095e,a55475b1,0.0,NaN,a55475b1,c7a5ad39,0,0,0.0,NaN,2.0,NaN,0.0,NaN,2017.0,NaN,a55475b1,ab3c25cf
96,409,8fd95e4b,a55475b1,0.0,NaN,a55475b1,c7a5ad39,0,0,0.0,NaN,2.0,NaN,0.0,NaN,2017.0,NaN,a55475b1,ab3c25cf
144,410,9a0c095e,a55475b1,0.0,NaN,a55475b1,c7a5ad39,0,0,NaN,NaN,2.0,NaN,NaN,NaN,2017.0,NaN,a55475b1,ab3c25cf
168,411,9a0c095e,a55475b1,0.0,NaN,a55475b1,c7a5ad39,0,0,0.0,NaN,2.0,NaN,0.0,NaN,2017.0,NaN,a55475b1,ab3c25cf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18721883,2700529,9a0c095e,8fd95e4b,0.0,0.0,c7a5ad39,c7a5ad39,0,0,NaN,0.0,2.0,2.0,NaN,0.0,2019.0,2019.0,ab3c25cf,ab3c25cf
18721931,2700530,a55475b1,9a0c095e,NaN,0.0,c7a5ad39,a55475b1,0,0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2006.0,ab3c25cf,a55475b1
18722291,2700531,8fd95e4b,8fd95e4b,0.0,0.0,3cbe86ba,c7a5ad39,1,0,NaN,NaN,2.0,2.0,NaN,NaN,2020.0,2007.0,ab3c25cf,ab3c25cf
18722819,2700532,8fd95e4b,9a0c095e,0.0,0.0,c7a5ad39,c7a5ad39,0,0,NaN,NaN,2.0,2.0,NaN,NaN,2019.0,2014.0,ab3c25cf,ab3c25cf


In [126]:
show_misses(df)

collater_valueofguarantee_1124L = 8.76 %
collater_valueofguarantee_876L = 27.43 %
pmts_dpd_1073P = 93.73 %
pmts_dpd_303P = 96.82 %
pmts_month_158T = 8.76 %
pmts_month_706T = 26.98 %
pmts_overdue_1140A = 93.71 %
pmts_overdue_1152A = 96.81 %
pmts_year_1139T = 8.76 %
pmts_year_507T = 26.98 %


- collater_typofvalofguarant, collater_valueofguarantee не информативные признаки, 90% приходится на одно значение, остальные не заполнены.
- 'pmts_month', 'pmts_year' в принципе не особо информативны.

#### credit_bureau_b_2

In [128]:
df = pd.read_parquet(TRAIN_PREFIX + "credit_bureau_b_2.parquet")
df.describe()

,case_id,num_group1,num_group2,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
count,1.286755e+06,1.286755e+06,1.286755e+06,1.281394e+06,1.281394e+06
mean,1.229444e+06,7.467404e-01,1.231973e+01,2.437045e+04,1.185945e+01
std,6.799923e+05,1.121661e+00,1.001713e+01,5.747955e+05,4.551762e+02
min,4.670000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.418980e+05,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00
50%,1.416105e+06,0.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00
75%,1.781534e+06,1.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00
max,2.703436e+06,2.000000e+01,3.600000e+01,1.851242e+08,1.474706e+05


In [129]:
df.describe(include=object)

,pmts_date_1107D
count,1286755
unique,58
top,2019-04-15
freq,49266


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1286755 entries, 0 to 1286754
Data columns (total 6 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   case_id                1286755 non-null  int64  
 1   num_group1             1286755 non-null  int64  
 2   num_group2             1286755 non-null  int64  
 3   pmts_date_1107D        1286755 non-null  object 
 4   pmts_dpdvalue_108P     1281394 non-null  float64
 5   pmts_pmtsoverdue_635A  1281394 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 58.9+ MB


In [131]:
df

,case_id,num_group1,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
0,467,0,0,2018-11-15,NaN,NaN
1,467,0,1,2018-12-15,NaN,NaN
2,467,1,0,2018-12-15,NaN,NaN
3,467,2,0,2016-10-15,0.0,0.0
4,467,2,1,2016-11-15,0.0,0.0
...,...,...,...,...,...,...
1286750,2703436,1,31,2020-05-15,0.0,0.0
1286751,2703436,1,32,2020-06-15,0.0,0.0
1286752,2703436,1,33,2020-07-15,0.0,0.0
1286753,2703436,1,34,2020-08-15,0.0,0.0


In [132]:
show_misses(df)

pmts_dpdvalue_108P = 0.42 %
pmts_pmtsoverdue_635A = 0.42 %


Далее после соединения с базовой таблицей доля пропусков будет 97.61 %, поэтому эта таблица игнорируется.

### Вывод

Основная таблица - base, к ней левым соединением добавим остальные таблицы. Таблицы с уровнем depth=1,2 перед соединением агрегируем по максимуму, т.к. там хранится историческая информация по каждому case_id.
Часть колонок имеет значительную долю пропусков, их исключим. Колонки с незначительными пропусками заполним медианой, модой. Категориальные данные обработаем. Из-за большого объема данных работать будем в polars.

- Перед соединением таблиц сократим количество колонок с 488 до 295 просто объединив похожие по смыслу.
- Далее в объединенной таблице сократим кол-во признаков с помощью анализа корреляции и доли пропусков.

# Промежуточные выводы

Проведен разведочный анализ данных.
- С помощью обзора таблиц и анализа корреляции выявлены взаимосвязи между признаками, что помогло объдинить их по смыслу и значениям.
- Удалили признаки с высокой долей пропусков.